In [1]:
print ('Missing Munna Bhaiyya')

Missing Munna Bhaiyya


In [2]:
import torch

In [3]:
words = open('names.txt','r').read().splitlines()

In [4]:
#Define a 27*27 array as this is a bigram modeal
N = torch.zeros((27,27), dtype = torch.int32)

In [5]:
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
#stoi['<E>'] = 27

In [6]:
for w in words:
    chs = ['.'] + list (w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1,ix2] +=1

In [7]:
P = (N+1).float()
P /= P.sum(1, keepdims=True)

In [8]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

junide.
janasah.
p.
cony.
a.


In [9]:
#Model Evaluation using log likelyhood
#1 Print bigrams of first 3 words in the dataset
#3 find prob of those bigrams in the trained model
#find logprob and negative logprob, eventually to define a loss function that we try to minimize by manupilating the weights

log_likelihood = 0.0
n = 0
for w in words:
#for w in ["srikar"]:
    chs = ['.']+list(w)+['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        log_prob = torch.log(prob)
        log_likelihood += log_prob
        n +=1
        #print(f'{ch1}{ch2}:{log_prob:.4f}:{log_likelihood=}')
nll = - log_likelihood
avg_nll = nll/n
print(f'{avg_nll=}')


avg_nll=tensor(2.4544)


In [10]:
# Moving into Neural net approach

In [11]:
#Create a training set for the neural net
#generate tuples from the word list. Get the corresponding integer for a given letter from strinToInt stoic matrix
#store them in xs and ys arrays

xs, ys = [], []
for w in words[:1]:
    chs = ['.'] + list (w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [12]:
#Forward Pass
#Use one-hot encoding to convert the xs into a float type matrix

import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes = 27).float()#

#Randomise weights for the inputs neurons
g = torch.Generator().manual_seed(2147483647)
W = torch.randn(27,27, generator=g, requires_grad = True)


In [13]:
for i in range(2):
    #Forward pass
    logits = xenc @ W #these are log of count of N matrix
    counts = logits.exp() #Taking exponential function will give us the actual counts
    probs = counts/counts.sum(1, keepdims = True) #normalize the counts to get the probability
    
    #Last two steps are called softmax. It takes any integers tensor as input and gives a probabilites as output
    
    #Get loss function
    #We use logloss as this is a classifaction type problem
    
    #1 get the probabilites of ys in the current model
    yhat = probs[torch.arange(5),ys]
    #2 get log of yhat
    yhatlog = yhat.log()
    #3 loss is the sum of negative log likelihood
    loss = -yhatlog.sum()
    print(f'{loss=}')
    
    #backwardpass
    W.grad = None
    loss.backward()
    
    #upgrade the Weigth by slightly nudging the wieghts by 0.1
    W.data = -8*W.grad  #-0.1 beacuse we want to decrease the loss function

loss=tensor(18.8465, grad_fn=<NegBackward0>)
loss=tensor(1.5580, grad_fn=<NegBackward0>)


In [25]:
#drawing letters from the bi-gram neural net model
out = [];
ix = 0;
while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes = 27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts/counts.sum(1, keepdims = True)
    ix = torch.multinomial(probs, num_samples = 1, replacement = True).item()
    out.append(itos[ix])
    print(out)
    if ix == 0:
        break

print(out)

['x']
['x', 't']
['x', 't', 'w']
['x', 't', 'w', 'm']
['x', 't', 'w', 'm', 'x']
['x', 't', 'w', 'm', 'x', 'c']
['x', 't', 'w', 'm', 'x', 'c', 'n']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd', 'x']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd', 'x', 'm']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd', 'x', 'm', 'x']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd', 'x', 'm', 'x', 'l']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd', 'x', 'm', 'x', 'l', 'n']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd', 'x', 'm', 'x', 'l', 'n', 'l']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd', 'x', 'm', 'x', 'l', 'n', 'l', 'w']
['x', 't', 'w', 'm', 'x', 'c', 'n', 'r', 'x', 'i', 'd', 'x', 'm', 'x', 'l', 'n', 'l', 'w', 'w']
['x', 't', 'w', 'm', 'x', 'c', 

In [ ]:
#backwardpass
W.grad = None
loss.backward()